In [21]:
import yfinance as yf
import pandas as pd
import datetime as dt

In [22]:
asset = "TSLA"

In [23]:
intraday = yf.download(asset, start="2023-12-01", end="2023-12-08", interval="1m")

[*********************100%%**********************]  1 of 1 completed


In [24]:
intraday

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2023-12-01 09:30:00-05:00,234.100006,234.160004,234.100006,234.160004,234.160004,4292380
2023-12-01 09:31:00-05:00,234.160004,235.759995,234.060104,235.059998,235.059998,1093234
2023-12-01 09:32:00-05:00,235.050507,235.100006,233.500000,233.990005,233.990005,931086
2023-12-01 09:33:00-05:00,233.981598,234.199997,232.289993,233.250000,233.250000,1321894
2023-12-01 09:34:00-05:00,233.210007,233.210007,231.910004,232.970001,232.970001,1170471
...,...,...,...,...,...,...
2023-12-07 15:55:00-05:00,242.729996,242.858597,242.520004,242.534195,242.534195,321565
2023-12-07 15:56:00-05:00,242.550003,242.649994,242.440002,242.557007,242.557007,258690
2023-12-07 15:57:00-05:00,242.539993,242.589996,242.300003,242.470093,242.470093,284621


In [30]:
def intraday_trend(df, entry, exit):
    ret_120min = df.iloc[120].Open/df.iloc[0].Open - 1
    tick_ret = df.Open.pct_change()
    if ret_120min > entry:
        buy_price = df.iloc[121].Open
        buy_time = df.iloc[121].name
        cumulated = (tick_ret.loc[buy_time:] + 1).cumprod() - 1
        exit_time = cumulated[(cumulated < -exit) | (cumulated > exit)].first_valid_index()
        if exit_time == None:
            exit_price = df.iloc[-1].Open
        else:
            exit_price = df.loc[exit_time + dt.timedelta(minutes=1)].Open
        profit = exit_price - buy_price
        profit_rel = profit/buy_price
        return profit_rel
    else:
        return None
        

In [31]:
intraday_trend(intraday, 0.01, 0.01)

0.013257305580109348

In [44]:
datesframe = yf.download(asset, start="2023-11-10", end="2023-12-08")

[*********************100%%**********************]  1 of 1 completed


In [45]:
datesframe.index

DatetimeIndex(['2023-11-10', '2023-11-13', '2023-11-14', '2023-11-15',
               '2023-11-16', '2023-11-17', '2023-11-20', '2023-11-21',
               '2023-11-22', '2023-11-24', '2023-11-27', '2023-11-28',
               '2023-11-29', '2023-11-30', '2023-12-01', '2023-12-04',
               '2023-12-05', '2023-12-06', '2023-12-07'],
              dtype='datetime64[ns]', name='Date', freq=None)

In [55]:
frames = []

for i in datesframe.index:
    frames.append(yf.download(asset, start=i, end=i + dt.timedelta(days=1), interval = "1m"))

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [58]:
returns = []

for i in frames: 
    returns.append(intraday_trend(i, 0.01, 0.02))

returns

[None,
 0.0019473869439003916,
 None,
 -0.02120268909307613,
 None,
 -0.006130322566791961,
 None,
 -0.002468811059122972,
 None,
 -0.006761113210003916,
 None,
 0.02085203143244704,
 None,
 None,
 0.009517713006828916,
 None,
 -0.018664081677675304,
 None,
 None]

In [59]:
pd.DataFrame(returns).mean()

0   -0.002864
dtype: float64